<a href="https://colab.research.google.com/github/ebahtiyar/Checking-and-Correcting-Grammatical-Errors-in-Turkish-with-Machine-Learning-and-Deep-Learning/blob/main/deda_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

^C
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,"/content/drive/My Drive/project")
import os
from sklearn.model_selection import train_test_split
import numpy as np
import random as r
import punc_preprocessing as punc
import pickle
import warnings
import gensim
from gensim.models import Word2Vec
import functions as f
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re
from keras.models import load_model
def replace_str_index(text,index=0,replacement=''):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])

Mounted at /content/drive


In [ ]:
def find_de_da_ki():
    data = punc.load_data("/content/drive/My Drive/project/database.db","Total_senteces_F1")
    c_data = []
    for line in data:
        words = line.split()
        if len(words) < 35:
          c_data.append(line)
    da_de = []
    ki = []
    for i in c_data:
        i = i.replace("’","")
        i = re.sub(r'[^\w\s]', '', i)
        try:
            if i.find("da") != -1 :
                index  = i.find("da")
                if i[index + 2] == " " and i[index-1] != "’":
                    da_de.append(i)
            if i.find("de") != -1:
                index = i.find("de")
                if i[index + 2] == " " and i[index-1] != "’":
                    da_de.append(i)
            if i.find("ki") != -1:
                index = i.find("ki")
                if i[index + 2] == " " and i.find("iki") == -1 and i.find("eski") == -1:
                    ki.append(i)
                                      
        except:
            pass
    return da_de,ki

In [ ]:
def data_simplify_conjunction():
    da_de,ki = find_de_da_ki()
    
    sp_da_de = []
    sp_ki = []
    
    adj_da_de = []
    adj_ki = []
    
    
    for i in da_de:
        try:
            if i.find("da") != -1:
                index = i.find("da")
                if i[index-1] == " ":
                    sp_da_de.append(i)
                else:
                    adj_da_de.append(i)
            if i.find("de") != -1:
                index = i.find("de")
                if i[index-1] == " ":
                    sp_da_de.append(i)
                else:
                    adj_da_de.append(i)           
        except:
            pass
        
    for i in ki:
        try:
            if i.find("ki") != -1:
                index = i.find("ki")
                if i[index-1] == " ":
                    sp_ki.append(i)
                else:
                    adj_ki.append(i)         
        except:
            pass
        
    return sp_da_de,sp_ki,adj_da_de,adj_ki  

In [ ]:
def generator_training_input(data):
    training_input = []
    training_label = []
    for line in data:
        line = line.strip()
        training_input.append(line)
        r_words = word_tokenizer(line)
        label = ""
        for word in r_words:
            label = label + "O "
        training_label.append(label)
        
        b_da = [m.start() for m in re.finditer('da ', line)]
        k  = 0
        for pos in b_da:
            if pos > 0:
                pos = pos + k
                if line[pos-1] == " ":
                    line = replace_str_index(line,index=pos-1,replacement='')
                    k = k - 1
                        
                elif line[pos-1].isalpha():
                    line = replace_str_index(line,index=pos,replacement=' d')
                    k = k + 1
                
        b_de = [m.start() for m in re.finditer('de ', line)]
        k  = 0     
        for pos in b_de:
            if pos > 0:
                pos = pos + k
                if line[pos-1] == " ":
                    line = replace_str_index(line,index=pos-1,replacement='')
                    k = k - 1
                        
                elif line[pos-1].isalpha():
                    line = replace_str_index(line,index=pos,replacement=' d')
                    k = k + 1
                    
        training_input.append(line)
        label1 = generator_training_output(line)
        training_label.append(label1)
        
    return training_input,training_label

In [ ]:
def generator_training_output(line):
    
        label = ""
        r_words = word_tokenizer(line)
                    
        for word in r_words:
            if word.find("da") > 0 or word.find("de") > 0:
                if len(word) > 1:
                    if word[len(word) - 2] == "d":
                        label = label + "E "
                    else:
                         label = label + "O "
                else:
                    label = label + "O "
            else:
                label = label + "O "
        label = label.strip()
        
          
        return label

In [ ]:
def word_tokenizer(line):
        line = re.sub(r'[^\w\s]', '', line)
        words = line.split()
        r_words = []
        try:
            for pos,word in enumerate(words):
                n = ""
                if pos > 0:
                   if word == "de":
                      before_word = words[pos-1]
                      n = before_word + " de"
                      r_words.remove(before_word)
                      r_words.append(n)
                   elif word == "da":
                        before_word = words[pos-1]
                        n = before_word + " da"
                        r_words.remove(before_word)
                        r_words.append(n)
                           
                   else:
                       r_words.append(word)
                else:
                    r_words.append(word)
        except:
            pass
                
        return r_words

In [ ]:
def createTokenizer(data,Idx):
  output = []
  for line in data:
      wordIndices = []
      words = word_tokenizer(line)
      for word in words:
        if word in Idx:
          wordIdx = Idx[word].index
        elif word.lower() in Idx:
          wordIdx = Idx[word.lower()].index
        else:
          wordIdx = Idx['UNK'].index
        wordIndices.append(wordIdx)
      output.append(wordIndices)
  return output

In [ ]:
def createTokenizer1(data,Idx):
  output = []
  for line in data:
      wordIndices = []
      words = word_tokenizer(line)
      for word in words:
        if word in Idx:
          wordIdx = Idx[word]
        elif word.lower() in Idx:
          wordIdx = Idx[word.lower()]
        else:
          wordIdx = Idx['UNK']
        wordIndices.append(wordIdx)
      output.append(wordIndices)
  return output

In [ ]:
def lastControl(pred_sentence):
    word_list = []
    r_words = word_tokenizer(pred_sentence)
    sentence = ""
    for pos,word in enumerate(r_words):
        if word == "dede" or word == "dada":
          if pos > 0:
              if word == "dede":
                before_word = r_words[pos-1]
                new_word = before_word + "de" + " de"
                word_list.append(new_word)
                word_list.remove(before_word)
              elif word == "dada":
                  before_word = r_words[pos-1]
                  new_word = before_word + "da" + " da"
                  word_list.append(new_word)
                  word_list.remove(before_word)
              else:
                word_list.append(word)
          else:
              word_list.append(word)
        else:
          word_list.append(word)
    for word in word_list:
        sentence = sentence + word + " "
    return sentence.lower()

In [ ]:
sp_da_de,sp_ki,adj_da_de,adj_ki = data_simplify_conjunction()
input_data = adj_da_de + sp_da_de
input_data = f.unique(input_data)
len(input_data)
training_input,training_label = generator_training_input(input_data)
print("Total Sentences: " + str(len(training_input)))

x_train , x_test , y_train , y_test = train_test_split(training_input,training_label,test_size=0.1)
print("Total Training Sentences: " + str(len(x_train)))

Total Sentences: 2081226
Total Training Sentences: 1873103


In [ ]:
import pandas as pd 
s = {"Traininig Sentences":training_input, "Labels": training_label }
s_df = pd.DataFrame(data = s)

In [ ]:
word_tokenizer(sp_da_de[1121]) # 1141

['Belki de', 'bu', 'yüzden', 'filmlerim', 'insanlara', 'dokunuyor']

In [ ]:
s_df.sample(5)

,Traininig Sentences,Labels
1276172,Müteahhitler arasında meydana gelecek sıkıntıl...,O O O O O O O O O O O O O O
165029,Annemde babama aşık oluyor iki oğlu zaten var ...,E O O O O O O O E O O O
2066808,Şirkete döndük çalışıyoruz annemle de görüşüyoruz,O O O O O
1893081,Önümüzdeki haftalar da çok özel bir kütüphaned...,O E O O O E O
339931,Benim bulunduğum Beyoğlu ilçesin de TP plakası...,O O O E O O O


In [ ]:
len(input_data)

1040613

In [ ]:
j = sp_ki + adj_ki

In [ ]:
227216*2

454432

In [ ]:
j = f.unique(j)
print(len(j))

227216


In [ ]:
sp_ki[32]

'Ama gelin görün ki dünyanın adaleti böyle tecelli etmiyor'

In [ ]:
traning_token =[]
for i in training_input:
  i = i.lower()
  traning_token.append(word_tokenizer(i))
traning_token.append(["UNK","PAD"])

In [ ]:
Word2Vec_model = Word2Vec(min_count=1)
Word2Vec_model.build_vocab(traning_token) 
Word2Vec_model.train(traning_token, total_examples=Word2Vec_model.corpus_count, epochs= 100)

(2320832462, 2466378200)

In [ ]:
Word2Vec_model.save("/content/drive/MyDrive/project/savedd_model/Word2Vec_model")

In [ ]:
pretrained_weights = Word2Vec_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [ ]:
x_train_tokenizer = createTokenizer(x_train,Word2Vec_model.wv.vocab)

labelSet = set()
for line in y_train:
  words = word_tokenizer(line)
  for word in words:
    labelSet.add(word.lower())

sorted_labels = sorted(list(labelSet), key=len)
# Create mapping for labels
label2Idx = {}
if len(label2Idx) == 0:
  label2Idx["PAD"] = len(label2Idx)
  label2Idx["UNK"] = len(label2Idx)
for label in sorted_labels:
  label2Idx[label] = len(label2Idx)
idx2Label = {v: k for k, v in label2Idx.items()}

y_train_tokenizer = createTokenizer1(y_train,label2Idx)

In [ ]:
max_in_len = int(len(max(x_train_tokenizer,key=len)))
max_out_len = int(len(max(x_train_tokenizer,key=len)))
x_train_pad = pad_sequences(x_train_tokenizer, max_in_len, padding = "post")
y_train_pad = pad_sequences(y_train_tokenizer, max_out_len, padding = "post")

x_train_pad = x_train_pad.reshape(*x_train_pad.shape, 1)
y_train_pad = y_train_pad.reshape(*y_train_pad.shape, 1)
print("Max Length of Train Sentences:" + str(max_in_len) + " Max Length of Train Labels:" + str(max_out_len))

Max Length of Train Sentences:34 Max Length of Train Labels:34


In [ ]:
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.layers import Dense, Activation , Input,RepeatVector, TimeDistributed,Bidirectional
from keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [ ]:
model_main = Sequential()
model_main.add(Input(shape=(max_in_len,)))
model_main.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size,weights=[pretrained_weights]))
model_main.add(Bidirectional(LSTM(units=emdedding_size,return_sequences=False)))
model_main.add(RepeatVector(max_out_len))
model_main.add(Bidirectional(LSTM(emdedding_size, return_sequences=True, dropout=0.1)))
model_main.add(TimeDistributed(Dense(len(label2Idx)+1)))
model_main.add(Activation('softmax'))
model_main.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])

In [ ]:
filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
callbacks = [checkpoint]

In [ ]:
model_main.fit(x_train_pad, y_train_pad, validation_split = 0.2, epochs= 3 , callbacks=callbacks, batch_size = 256)

In [ ]:
model_main.save('/content/drive/My Drive/project/savedd_model/model_dade_w2v_3.h5')#v_2 0.01077 best

In [ ]:
model_main  = load_model('/content/drive/My Drive/project/savedd_model/model_dade_w2v_3.h5')

In [ ]:
idx2Label = {0 :'PAD', 1:'UNK' , 2: 'o' , 3: 'e'}
Word2Vec_model=Word2Vec.load("/content/drive/MyDrive/project/savedd_model/Word2Vec_model")

In [ ]:
def prediction(model_main,sample,Word2Vec_model,idx2Label):
  sample_pad = []
  words = word_tokenizer(sample.lower())
  for word in words:
      if word in Word2Vec_model.wv.vocab:
          
         sample_pad.append(Word2Vec_model.wv.vocab[word].index)
      else:
         sample_pad.append(Word2Vec_model.wv.vocab["UNK"].index)

  sample_pad = pad_sequences([sample_pad], maxlen=34, padding='post')

  pred = model_main.predict(sample_pad)
  label = [idx2Label[np.argmax(x)] for x in pred[0]]
  output_label = ""
  for i in label:
      output_label = output_label + i + " "
  

  
  pred_sent = ""
  for i in zip(words,label):
    if i[1] == "o":
      pred_sent = pred_sent +  i[0] + " "
    elif i[1] == "e":
        if i[0].find(" de") > 0:
            new_word = replace_str_index(i[0],index=i[0].find(" de"),replacement='')
            pred_sent = pred_sent + new_word + " "
        elif i[0].find(" da") > 0:
            new_word = replace_str_index(i[0],index=i[0].find(" da"),replacement='')
            pred_sent = pred_sent + new_word + " "
        elif i[0].find(" ") == -1 and i[0].find("da") == len(i[0]) - 2:
            new_word = replace_str_index(i[0],i[0].find("da"),replacement=' d')
            pred_sent = pred_sent + new_word + " "
        elif i[0].find(" ") == -1 and i[0].find("de") == len(i[0]) - 2:
            new_word = replace_str_index(i[0],i[0].find("de"),replacement=' d')
            pred_sent = pred_sent + new_word + " "
        else:
            pred_sent = pred_sent + i[0] + " "
            
    elif i[0][1] == "PAD":
        continue
  pred_sent = pred_sent.strip()

  r_words = word_tokenizer(pred_sent)


  return pred_sent

In [ ]:
sample = "Ve meclislerin de artık her biri başkan olabilecek kadar yıldız isimlerden olması gerekir"

In [ ]:
prediction(model_main,sample.lower(),Word2Vec_model,idx2Label).strip()

've meclislerinde artık her biri başkan olabilecek kadar yıldız isimlerden olması gerekir'

In [ ]:
def generator_testing_input(data):
    training_input = []
    training_label = []
    for line in data:
        line = line.strip()
        b_da = [m.start() for m in re.finditer('da ', line)]
        k  = 0
        for pos in b_da:
            if pos > 0:
                pos = pos + k
                if line[pos-1] == " ":
                    line = replace_str_index(line,index=pos-1,replacement='')
                    k = k - 1
                        
                elif line[pos-1].isalpha():
                    line = replace_str_index(line,index=pos,replacement=' d')
                    k = k + 1
                
        b_de = [m.start() for m in re.finditer('de ', line)]
        k  = 0     
        for pos in b_de:
            if pos > 0:
                pos = pos + k
                if line[pos-1] == " ":
                    line = replace_str_index(line,index=pos-1,replacement='')
                    k = k - 1
                        
                elif line[pos-1].isalpha():
                    line = replace_str_index(line,index=pos,replacement=' d')
                    k = k + 1
                    
        training_input.append(line)
    
    return training_input

In [ ]:
def find_de_da_ki1():
    data = punc.load_data("/content/drive/My Drive/project/test.db","test_sentences")
    c_data = []
    for line in data:
        words = line.split()
        if len(words) < 35:
          c_data.append(line)
    da_de = []
    ki = []
    for i in c_data:
        i = i.replace("’","")
        i = re.sub(r'[^\w\s]', '', i)
        try:
            if i.find("da") != -1 :
                index  = i.find("da")
                if i[index + 2] == " " and i[index-1] != "’":
                    da_de.append(i)
            if i.find("de") != -1:
                index = i.find("de")
                if i[index + 2] == " " and i[index-1] != "’":
                    da_de.append(i)
            if i.find("ki") != -1:
                index = i.find("ki")
                if i[index + 2] == " " and i.find("iki") == -1 and i.find("eski") == -1:
                    ki.append(i)
                                      
        except:
            pass
    return da_de,ki

In [ ]:
def data_simplify_conjunction1():
    da_de,ki = find_de_da_ki1()
    
    sp_da_de = []
    sp_ki = []
    
    adj_da_de = []
    adj_ki = []
    
    
    for i in da_de:
        try:
            if i.find("da") != -1:
                index = i.find("da")
                if i[index-1] == " ":
                    sp_da_de.append(i)
                else:
                    adj_da_de.append(i)
            if i.find("de") != -1:
                index = i.find("de")
                if i[index-1] == " ":
                    sp_da_de.append(i)
                else:
                    adj_da_de.append(i)           
        except:
            pass
        
    for i in ki:
        try:
            if i.find("ki") != -1:
                index = i.find("ki")
                if i[index-1] == " ":
                    sp_ki.append(i)
                else:
                    adj_ki.append(i)         
        except:
            pass
        
    return sp_da_de,sp_ki,adj_da_de,adj_ki  

In [ ]:
sp_da_de_t,sp_ki_t,adj_da_de_t,adj_ki_t  = data_simplify_conjunction1()

In [ ]:
len(sp_da_de_t),len(adj_da_de_t)

(27735, 63345)

In [ ]:
test = r.choices(sp_da_de_t,k = 100) + r.choices(adj_da_de_t , k = 100)
test = f.unique(test)
r.shuffle(test)
testing_input = generator_testing_input(test)

In [ ]:
len(test)

10046

In [ ]:
pred = []
pred_last = []
wrong = []
wrong_pos = []
for line in testing_input:
    pred.append(prediction(model_main,line.lower(),Word2Vec_model,idx2Label).strip())

for line in pred:
    if (line.find(" dede ") > 0) or (line.find(" dada ") > 0):
       line = lastControl(line)
    pred_last.append(line.strip())
   
 
k = 0
for pos,line in enumerate(test):
    line = line.lower()
    line = line.replace("i̇","i")
    if line.lower() == pred_last[pos]:
        k = k + 1
    else:
      wrong.append(pred_last[pos])
      wrong_pos.append(pos)
        

In [ ]:
rand = r.randrange(100)
print("Score: " + str(k) +" in " + str(len(testing_input)) + " sentences")
print("Orginal: "+test[rand].lower())
print("Wrong: "+testing_input[rand].lower())
print("Prediction: "+pred[rand])

Score: 182 in 199 sentences
Orginal: o poliklinik de çok başarılı oldu
Wrong: o poliklinikde çok başarılı oldu
Prediction: o poliklinik de çok başarılı oldu


In [ ]:
def prediction_score(model_main,sample,Word2Vec_model,idx2Label):
  sample_pad = []
  words = word_tokenizer(sample.lower())
  for word in words:
      if word in Word2Vec_model.wv.vocab:
          
         sample_pad.append(Word2Vec_model.wv.vocab[word].index)
      else:
         sample_pad.append(Word2Vec_model.wv.vocab["UNK"].index)

  sample_pad = pad_sequences([sample_pad], maxlen=34, padding='post')

  pred = model_main.predict(sample_pad)
  label = [idx2Label[np.argmax(x)] for x in pred[0]]

  return label

In [ ]:
pred_label = []

for line in testing_input:
    pred_label.append(prediction_score(model_main,line.lower(),Word2Vec_model,idx2Label))


pred_label_l = []

for pred in pred_label:
    t = []
    for i in pred:
        if i != "PAD":
           if i == "e":
              i = "D_DEC"
           elif i == "o":
              i = "C_CON"
           t.append(i.upper())
        else:
          break
    pred_label_l.append(t)

In [ ]:
test_label = []

for line in testing_input:
    test_label.append(generator_training_output(line).lower())
test = []
for line in test_label:
     line = line.upper()
     x =  []
     for l in line.split():
         if l == "O":
            i = "D_CON"
         else:
            i = "D_DEC"
         x.append(i)
     test.append(x)

test_x = []
text_label = []
for i in range(len(test)):
  if len(test[i]) == len(pred_label_l[i]):
      test_x.append(test[i])
      text_label.append(pred_label_l[i]) 

In [ ]:
from seqeval.metrics import classification_report, f1_score

In [ ]:
score = classification_report(test_x,text_label)
print(score)

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: D_CON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: D_DEC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: C_CON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

        _CON       0.84      0.83      0.83     22781
        _DEC       0.87      0.87      0.87     14978

   micro avg       0.85      0.85      0.85     37759
   macro avg       0.86      0.85      0.85     37759
weighted avg       0.85      0.85      0.85     37759

